<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Integration_and_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import joblib
import os
import numpy as np
import pandas as pd
pd.options.display.max_rows = 9999
!pip install autorch > log.txt
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import random
random.seed(11)
np.random.seed(11)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# columns name

In [3]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')
print(icg_c.keys())
print(c620_c.keys())
print(c660_c.keys())
print(c670_c.keys())
print(t651_c.keys())

dict_keys(['x', 'y'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['xTray43', 'xTray54', 'combined', 'upper_bf', 'lower_bf', 'yRefluxRate', 'yControl', 'yHeatDuty', 'distillate_x', 'bottoms_x', 'distillate_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'MFR'])


# DataFrame

In [4]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/icg_train.csv',index_col=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0)
idx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(idx)

1296

In [5]:
icg_df.loc[idx].head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Simulation Case Conditions_C620 Distillate Rate_m3/hr
004-008,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,72.0,0.4524,44.244236,44.786339,90.0,920.0,0.01
111-026,188.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,57.0,0.2127,35.413666,57.973106,70.0,920.0,0.01
036-017,175.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,92.0,0.2128,35.413631,57.973057,80.0,890.0,0.01
014-017,188.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,41.0,0.4524,44.244236,44.786339,80.0,920.0,0.01
100-008,163.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,72.0,0.2127,35.413666,57.973106,90.0,920.0,0.01


In [6]:
c620_df.loc[idx][c620_c['case']].head()

,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%
004-008,36.0,0.01,90.000000
111-026,36.0,0.01,70.000015
036-017,36.0,0.01,79.999992
014-017,36.0,0.01,80.000015
100-008,36.0,0.01,89.999992


In [7]:
c660_df.loc[idx][c660_c['case']].head()

,Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
004-008,920.004387,9.999987
111-026,920.009313,5.000001
036-017,889.992961,5.000018
014-017,920.003760,5.000975
100-008,920.002107,5.000043


# Input data

In [8]:
# icg
icg_input = icg_df.loc[idx,icg_c['x']]
icg_input = icg_input.join(c620_df.loc[idx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
icg_input = icg_input.join(c660_df.loc[idx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
icg_input = icg_input.join(c620_df.loc[idx].filter(regex='Receiver Temp'))

# c620
c620_feed = c620_df.loc[idx,c620_c['x41']]

# t651
t651_feed = t651_df.loc[idx,t651_c['x41']]

In [9]:
icg_input.head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
004-008,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,72.0,0.4524,44.244236,44.786339,90.0,920.0,0.01,9.999987,36.0
111-026,188.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,57.0,0.2127,35.413666,57.973106,70.0,920.0,0.01,5.000001,36.0
036-017,175.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,92.0,0.2128,35.413631,57.973057,80.0,890.0,0.01,5.000018,36.0
014-017,188.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,41.0,0.4524,44.244236,44.786339,80.0,920.0,0.01,5.000975,36.0
100-008,163.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,72.0,0.2127,35.413666,57.973106,90.0,920.0,0.01,5.000043,36.0


# Output data

In [10]:
c620_op = c620_df.loc[idx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[idx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[idx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[idx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[idx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[idx,c670_c['distillate_x']+c670_c['bottoms_x']]

# config

In [11]:
config = {
    
      'icg_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg_svr.pkl',
      'c620_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620.pkl',
      'c660_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660.pkl',
      'c670_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',

      # real data model path
      'icg_model_path_real_data':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg_svr_real_data.pkl',
      'c620_model_path_real_data':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_real_data.pkl',
      'c660_model_path_real_data':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_real_data.pkl',
      'c670_model_path_real_data':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670_real_data.pkl',
      
      # col_names
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      
      # Special column (0.9999 & 0.0001)
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',

      # sp
      'c620_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_wt_always_same_split_factor_dict.pkl',
      'c660_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_wt_always_same_split_factor_dict.pkl',
      'c670_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c670_wt_always_same_split_factor_dict.pkl',
          }

# define F

In [12]:
class F(object):
  def __init__(self,config):
    # simulation data model
    self.icg_model = joblib.load(config['icg_model_path'])
    self.c620_model = joblib.load(config['c620_model_path'])
    self.c660_model = joblib.load(config['c660_model_path'])
    self.c670_model = joblib.load(config['c670_model_path'])
    
    # real data model
    self.icg_real_data_model = joblib.load(config['icg_model_path_real_data'])
    self.c620_real_data_model = joblib.load(config['c620_model_path_real_data'])
    self.c660_real_data_model = joblib.load(config['c660_model_path_real_data'])
    self.c670_real_data_model = joblib.load(config['c670_model_path_real_data'])
    
    # columns name
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    
    # other infomation
    self.c620_wt_always_same_split_factor_dict = joblib.load(config['c620_wt_always_same_split_factor_dict'])
    self.c660_wt_always_same_split_factor_dict = joblib.load(config['c660_wt_always_same_split_factor_dict'])
    self.c670_wt_always_same_split_factor_dict = joblib.load(config['c670_wt_always_same_split_factor_dict'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749
    
    # user can set two mode
    self.Recommended_mode = False
    self.real_data_mode = False
    self._Post_processing = True

  def ICG_loop(self,Input):
    while True:
      if self.real_data_mode == True:
        output = pd.DataFrame(self.icg_real_data_model.predict(Input[self.icg_col['x']].values),
        index=Input.index,columns=['Simulation Case Conditions_C620 Distillate Rate_m3/hr'])
      if self.real_data_mode == False:
        output = pd.DataFrame(self.icg_model.predict(Input[self.icg_col['x']].values),
        index=Input.index,columns=['Simulation Case Conditions_C620 Distillate Rate_m3/hr'])
      dist_rate = output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0]
      na_in_benzene = Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
      print('current Distillate Rate_m3/hr:{} NA in Benzene_ppmw:{}'.format(dist_rate,na_in_benzene))
      if output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0] > 0:
        return output,Input
      else:
        Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 30
        print('NA in Benzene_ppmw -= 30')
  
  def __call__(self,icg_input,c620_feed,t651_feed):
    
    # get index
    idx = icg_input.index

    # c620_case 
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])

    # c620_case(Receiver Temp_oC) = user input
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    
    if self.Recommended_mode == True:
      icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = 980.0 
      icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = 70.0
      icg_output,icg_input = self.ICG_loop(icg_input)
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_output.values
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    
    if self.Recommended_mode == False:
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    
    # c620_input(c620_case&c620_feed)
    c620_input = c620_case.join(c620_feed)
    
    # c620 output(op&wt)
    c620_input = c620_case.join(c620_feed)
    c620_output = self.c620_model.predict(c620_input)
    c620_sp,c620_op = c620_output.iloc[:,:41*4],c620_output.iloc[:,41*4:]
    
    # update by c620 real data model?
    if self.real_data_mode == True:
      c620_op_real = self.c620_real_data_model.predict(c620_input).iloc[:,41*4:] #操作條件放後面
      c620_op.update(c620_op_real)
      c620_sp_real = self.c620_real_data_model.predict(c620_input).iloc[:,:41*4] #分離係數放前面
      c620_sp.update(c620_sp_real)
    
    # c620 sp後處理
    if self._Post_processing:
      for i in self.c620_wt_always_same_split_factor_dict.keys():
        c620_sp[i] = self.c620_wt_always_same_split_factor_dict[i]
    
    # 計算 c620_wt
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # c620_wt 後處理
    if self._Post_processing:
      bz_idx = c620_wt.columns.tolist().index('Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%')
      other_idx = [i for i in range(41*2,41*3,1) if i != bz_idx]
      other_total = (100 - c620_input['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'].values).reshape(-1,1)
      c620_wt.iloc[:,bz_idx] = c620_input['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'].values
      c620_wt.iloc[:,other_idx] = (c620_wt.iloc[:,other_idx].values /
                                   c620_wt.iloc[:,other_idx].values.sum(axis=1).reshape(-1,1))*other_total
    
    # c620 input mass flow rate m3 to ton
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    
    # c620 output mass flow ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)

    # t651 feed mass flow rate(ton)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)

    # c660 input mass flow(ton)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf

    # c660 input(feed & case)
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    
    if self.Recommended_mode == True:
      # fix Toluene in Benzene_ppmw = 10
      c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10.0
    
    if self.Recommended_mode == False:
      # Toluene in Benzene_ppmw = user input
      c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values
    
    c660_input = c660_case.join(c660_feed)
    
    # c660 output(op&wt)
    c660_output = self.c660_model.predict(c660_input)
    c660_sp,c660_op = c660_output.iloc[:,:41*4],c660_output.iloc[:,41*4:]

    # update by c660 real data model?
    if self.real_data_mode == True:
      c660_op_real = self.c660_real_data_model.predict(c660_input).iloc[:,41*4:] #操作條件放後面
      c660_op.update(c660_op_real)
      c660_sp_real = self.c660_real_data_model.predict(c660_input).iloc[:,:41*4] #分離係數放前面
      c660_sp.update(c660_sp_real)
    
    # c660 sp後處理
    if self._Post_processing:
      for i in self.c660_wt_always_same_split_factor_dict.keys():
        c660_sp[i] = self.c660_wt_always_same_split_factor_dict[i]
    
    # 計算 c660_wt
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # c660_wt 後處理
    if self._Post_processing:
      na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
      other_idx = list(set([*range(41)])-set(na_idx))
      na_total = (c660_input['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'].values/10000).reshape(-1,1)
      other_total = 100 - na_total
      c660_wt.iloc[:,41*2:41*3].iloc[:,na_idx] = (c660_wt.iloc[:,41*2:41*3].iloc[:,na_idx].values/
                                                  c660_wt.iloc[:,41*2:41*3].iloc[:,na_idx].values.sum(axis=1).reshape(-1,1))*na_total
      c660_wt.iloc[:,41*2:41*3].iloc[:,other_idx] = (c660_wt.iloc[:,41*2:41*3].iloc[:,other_idx].values/
                                                     c660_wt.iloc[:,41*2:41*3].iloc[:,other_idx].values.sum(axis=1).reshape(-1,1))*other_total
    
    # c660 output mass flow (ton)
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    
    # c670 input mass flow
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    
    # c670 feed wt%
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])

    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    
    # c670 input (feed%bf)
    c670_input = c670_feed.join(upper_bf)
    c670_output = self.c670_model.predict(c670_input)
    c670_sp,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]

    # update by c670 real data model?
    if self.real_data_mode == True:
      c670_op_real = self.c670_real_data_model.predict(c670_input).iloc[:,41*2:] #操作條件放後面
      c670_op.update(c670_op_real)
      c670_sp_real = self.c670_real_data_model.predict(c670_input).iloc[:,:41*2] #分離係數放前面
      c670_sp.update(c670_sp_real)
    
    # c670 sp後處理
    if self._Post_processing:
      for i in self.c670_wt_always_same_split_factor_dict.keys():
        c670_sp[i] = self.c670_wt_always_same_split_factor_dict[i]
    
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op

# 試算模式測試

In [13]:
f = F(config)
f.Recommended_mode = False
f.real_data_mode = False
f._Post_processing = True

In [14]:
commom_idx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
idx = np.random.choice(commom_idx,size=100,replace=False,p=None)

# minibatch input 
icg_input = icg_input.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]

# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]

# predict output

In [15]:
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f(icg_input,c620_feed,t651_feed)

# c620 show_metrics

In [16]:
f.c620_model.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.998783,2.80346e-05,0.157898
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.998862,0.00560461,0.115713
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.997499,0.000900209,0.0511588
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.999446,0.000923262,0.260447
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Pentane_wt%,0.999309,0.000754145,0.637907
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Hexane_wt%,0.996901,4.81835e-06,0.924687
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Benzene_wt%,0.990223,0.00149889,0.382162
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Cyclohexane_wt%,0.990067,6.26306e-09,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Heptane_wt%,0.996775,6.97209e-10,NaN


In [17]:
f.c620_model.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999958,6.16827e-11,0.000768895
Density_Vent Gas Production Rate and Composition,0.996545,8.11662e-08,0.0528195
Density_Distillate Production Rate and Composition,0.997983,2.25054e-07,0.04238
Density_Sidedraw Production Rate and Composition,0.999771,2.22927e-10,0.00134649
Density_Bottoms Production Rate and Composition,0.999644,5.85976e-12,0.000196919
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.999493,0.0781588,0.158655
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.99963,0.000472813,0.121555
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.99958,0.000771667,0.150079
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.999545,0.000494317,0.00870018
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.999641,0.000266399,0.00609045


# c660 show_metrics

In [18]:
f.c660_model.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.932512,0.0154465,4.8985
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.99055,0.41681,1.95036
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.950002,0.333312,1.69055
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.993744,0.084373,3.75154
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Pentane_wt%,0.991166,0.148373,5.88872
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Hexane_wt%,0.993113,0.00273469,3.71519
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Benzene_wt%,0.943788,0.543929,1.35088
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Cyclohexane_wt%,0.973959,3.35201e-06,1.57505
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Heptane_wt%,0.98782,1.77223e-07,NaN


In [19]:
f.c660_model.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.998375,1.1537e-09,0.00313743
Density_Vent Gas Production Rate and Composition,0.995251,1.75041e-06,0.179057
Density_Distillate (Benzene Drag) Production Rate and Composition,0.995644,3.81816e-07,0.054497
Density_Sidedraw (Benzene )Production Rate and Composition,0.975938,1.48339e-12,0.000111803
Density_Bottoms Production Rate and Composition,0.999271,5.66355e-11,0.000662503
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.997326,2.37031,0.902705
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.99706,0.0197878,0.874252
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.997389,0.0188133,0.884808
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.997141,0.000124265,0.00993402
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.997163,0.00218526,0.041499


# c670 show_metrics

In [20]:
f.c670_model.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,5.82043e-47,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Pentane_wt%,0.999459,3.08765e-36,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Hexane_wt%,0.999154,2.30154e-22,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Benzene_wt%,0.999873,2.41382e-07,0.118006
Toluene Column C670 Operation_Distillate Production Rate and Composition_Cyclohexane_wt%,0.999863,2.20048e-13,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Heptane_wt%,0.999842,8.09084e-11,NaN


In [21]:
f.c670_model.show_metrics(c670_op,c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.994404,5.07053e-12,0.000207718
Density_Bottoms Production Rate and Composition,0.998289,3.26979e-10,0.00139726
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.984426,10.181,0.988284
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.985338,0.136269,1.00055
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.983245,0.149003,1.0295
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.980102,0.00103634,0.0147105
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.998155,0.00517003,0.0218327
AVG,0.989137,1.49606,0.43664


# 推薦模式 測試

In [22]:
# change mode
f.Recommended_mode = True
f.Recommended_mode

True

In [23]:
# select one sample c620_side == 70 & NA in Benzene == 980
cond = (icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']==70)&(icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']==980)
sample = icg_input.sample(1)
sample.head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
073-008,163.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244236,44.786339,90.0,920.0,1.25,2.500002,36.0


In [24]:
idx = sample.index
print(idx)

Index(['073-008'], dtype='object')


In [25]:
demo = {
      # input
      'icg_input':icg_input.loc[idx],
      'c620_feed':c620_feed.loc[idx],
      't651_feed':t651_feed.loc[idx],
      # output
      'c620_op':c620_op.loc[idx],
      'c620_wt':c620_wt.loc[idx],
      'c660_op':c660_op.loc[idx],
      'c660_wt':c660_wt.loc[idx],
      'c670_op':c670_op.loc[idx],
      'c670_wt':c670_wt.loc[idx],
      }

In [26]:
icg_input = demo['icg_input'].copy()
c620_feed = demo['c620_feed'].copy()
t651_feed = demo['t651_feed'].copy()

In [27]:
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f(icg_input,c620_feed,t651_feed)

current Distillate Rate_m3/hr:0.3801330074803746 NA in Benzene_ppmw:980.0


In [28]:
for i in [c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_]:
  print(i.isnull().sum().sum())

0
0
0
0
0
0


In [29]:
demo['icg_input'].head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
073-008,163.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244236,44.786339,90.0,920.0,1.25,2.500002,36.0


In [30]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']